In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
import os
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import regex as re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import collections
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [88]:
lem = WordNetLemmatizer()

*(a) Carry out the suitable preprocessing steps on the given dataset.*

In [89]:
inverted_index = {}  #term-doc_name pairs

def preprocess_doc_and_get_terms(doc, doc_name):
    doc = doc.lower()
    doc = re.sub(r'[^\w\s]', ' ', doc)
    words = nltk.word_tokenize(doc)
    words = ["".join(ch for ch in word if ch.isalpha()) for word in words]
    words = {lem.lemmatize(word) for word in words if word not in set(stopwords.words("english"))}
    words = list(words)
    for j in range(len(words)):
        if words[j] in inverted_index.keys():
            inverted_index[words[j]].add(doc_name)
        else:
            inverted_index[words[j]] = {doc_name}
    return words

In [90]:
directory = os.fsencode("/content/drive/MyDrive/IR Assignments/Humor,Hist,Media,Food")
    
doc_text = []
doc_names = []
doc_preprocessed_words = []

for doc_name in os.listdir(directory):
    doc_path = os.path.join(directory,doc_name)
    doc = open(doc_path, encoding="utf-8", errors="surrogateescape")
    raw_doc = doc.read()
    doc_text.append(raw_doc)
    doc_names.append(doc_name)
    doc_preprocessed_words.append(preprocess_doc_and_get_terms(raw_doc,doc_name))

corpus = pd.DataFrame(list(zip(doc_names, doc_text, doc_preprocessed_words)), columns = ['doc_name', 'doc_text', 'doc_words'])
corpus.head()

,doc_name,doc_text,doc_words
0,b'talkbizr.txt',Newsgroups: talk.bizarre\nFrom: rigler@dao.nrc...,"[, fbi, followup, chaos, paper, gamepiece, ssc..."
1,b'takenote.jok',From davidt@psuhcx Tue Aug 2 22:30:02 1988\nF...,"[, form, said, psuhcx, ever, tue, lucidity, de..."
2,b'sysadmin.txt',From: szielins@us.oracle.com (szielins.US1)\nN...,"[, cobol, bit, jagermeister, biff, maniacal, b..."
3,b'taping.hum',\t\t\t [The TAPING I by: UNDERWAREZ.]\n\n ...,"[, wait, yeah, hmmm, pizza, jeepers, mind, ope..."
4,b'temphell.jok',The temperature of Heaven can be rather accura...,"[, vol, stefan, hotter, unbelieving, mean, sev..."


In [109]:
corpus.to_csv('corpus.csv', encoding="utf-8", errors="surrogateescape")

In [110]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1133 entries, 0 to 1132
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   doc_name   1133 non-null   object
 1   doc_text   1133 non-null   object
 2   doc_words  1133 non-null   object
dtypes: object(3)
memory usage: 26.7+ KB


In [111]:
print(len(inverted_index.keys()))

58479


In [112]:
# for k, v in inverted_index.items():
#     print(k, v)

*(b) Implement the unigram inverted index data structure.*

In [113]:
sorted_inverted_index = collections.OrderedDict(sorted(inverted_index.items()))

In [114]:
sorted_inverted_index = dict(sorted_inverted_index)  # POSTINGS LIST
# for k, v in sorted_inverted_index.items():
#     print(k, v)

In [115]:
len(sorted_inverted_index.keys())

58479

In [116]:
for key in sorted_inverted_index.keys():
    sorted_inverted_index[key] = sorted(sorted_inverted_index[key]) #sorted() returns a list

In [128]:
# print(sorted_inverted_index['world'])
# print(len(sorted_inverted_index['world']))

*(c) Provide support for the following queries-*
```
(i) x OR y
(ii) x AND y
(iii) x AND NOT y
(iv) x OR NOT y
```

In [118]:
def preprocess_query(query):
    query = query.lower()
    query = re.sub(r'[^\w\s]', ' ', query)
    words = nltk.word_tokenize(query)
    words = ["".join(ch for ch in word if ch.isalpha()) for word in words]
    words = {lem.lemmatize(word) for word in words if word not in set(stopwords.words("english"))}
    words = list(words)
    return words

In [119]:
def OR(docs1, docs2):
    docs = []
    comparisons = 0

    while(len(docs1) > 0 and len(docs2) > 0):
        if(docs1[0] < docs2[0]):
            comparisons = comparisons+1
            docs.append(docs1[0])
            docs1 = docs1[1:]
        elif(docs1[0] > docs2[0]):
            comparisons = comparisons+1
            docs.append(docs2[0])
            docs2 = docs2[1:]
        else:
            comparisons = comparisons+1
            docs.append(docs1[0])
            docs1 = docs1[1:]
            docs2 = docs2[1:]
    while(len(docs1) > 0):
        docs.append(docs1[0])
        docs1 = docs1[1:]
    while(len(docs2) > 0):
        docs.append(docs2[0])
        docs2 = docs2[1:]

    return docs, comparisons

def AND(docs1, docs2):
    docs = []
    comparisons = 0

    while(len(docs1) > 0 and len(docs2) > 0):
        if(docs1[0] > docs2[0]):
            comparisons = comparisons+1
            docs2 = docs2[1:]
        elif(docs1[0] < docs2[0]):
            comparisons = comparisons+1
            docs1 = docs1[1:]
        else:
            comparisons = comparisons+1
            docs.append(docs1[0])
            docs1 = docs1[1:]
            docs2 = docs2[1:]

    return docs, comparisons

def AND_NOT(docs1, docs2):
    common_docs, comparisons = AND(docs1.copy(), docs2.copy())
    docs = []

    while(len(docs1) > 0):
        if(len(common_docs) == 0):
            docs.append(docs1[0])
            docs1 = docs1[1:]
        elif(docs1[0] == common_docs[0]):
            docs1 = docs1[1:]
            common_docs = common_docs[1:]
            comparisons = comparisons+1
        elif(docs1[0] < common_docs[0]):
            docs.append(docs1[0])
            docs1 = docs1[1:]
            comparisons = comparisons+1

    return docs, comparisons

def OR_NOT(docs1,docs2):
    docs_not_having_2 = []
    comparisons_all = 0

    for a in corpus['doc_name']:
        if a not in docs2:
            docs_not_having_2.append(a)
            comparisons_all = comparisons_all+1
    docs_not_having_2 = sorted(docs_not_having_2)

    docs, comparisons_or = OR(docs1.copy(),docs_not_having_2.copy())

    return docs, comparisons_all + comparisons_or

In [124]:
def process_query(query, operators):
    total_comparisons = 0

    if(len(query) != len(operators)+1):
        print("Invalid Query!!!")
        return
    else:
        for i, op in enumerate(operators):
            if i == 0:
                if op == "AND":
                    docs, comparisons = AND(sorted_inverted_index[query[i]].copy(),sorted_inverted_index[query[i+1]].copy())
                elif op == "OR":
                    docs, comparisons = OR(sorted_inverted_index[query[i]].copy(),sorted_inverted_index[query[i+1]].copy())
                elif op == "AND NOT":
                    docs, comparisons = AND_NOT(sorted_inverted_index[query[i]].copy(),sorted_inverted_index[query[i+1]].copy())
                elif op == "OR NOT":
                    docs, comparisons = OR_NOT(sorted_inverted_index[query[i]].copy(),sorted_inverted_index[query[i+1]].copy())
                else:
                    print("Invalid Query!!!")
                    return
            else:
                if op == "AND":
                    docs, comparisons = AND(docs.copy(),sorted_inverted_index[query[i+1]].copy())
                elif op == "OR":
                    docs, comparisons = OR(docs.copy(),sorted_inverted_index[query[i+1]].copy())
                elif op == "AND NOT":
                    docs, comparisons = AND_NOT(docs.copy(),sorted_inverted_index[query[i+1]].copy())
                elif op == "OR NOT":
                    docs, comparisons = OR_NOT(docs.copy(),sorted_inverted_index[query[i+1]].copy())
                else:
                    print("Invalid Query!!!")
                    return
            total_comparisons += comparisons
            
    return docs, total_comparisons

*(d) Evaluation against some queries in the format given below*
```
Input Format:

The first line contains the number of queries, N.
The next 2N lines would represent the queries.
Each query would consist of two lines:
(a) line 1: Input sentence
(b) line 2: Input operation sequence

Output Format:

• The number of documents retrieved.
• The minimum number of total comparisons done (if any)( only in merging algorithm).
• The list of document names retrieved.
```

In [127]:
N = int(input("Enter the number of queries: "))

for _ in range(N):
    query = input("Enter input sequence: ")
    query = preprocess_query(str(query))
    operators = input("Enter operation sequence: ")
    operators = operators.upper().split(", ")
    # print(query)
    # print(operators)

    docs, comparisons = process_query(query, operators)
    
    print("Number of documents matched: ",len(docs))
    print("No. of comparisons required: ",comparisons)
    print("list of documents matched: ",docs)

Enter the number of queries: 2
Enter input sequence: lion stood peacefully for a moment
Enter operation sequence: or, or, or
Number of documents matched:  211
No. of comparisons required:  471
list of documents matched:  [b'a_tv_t-p.com', b'aeonint.txt', b'allusion', b'ambrose.bie', b'anim_lif.txt', b'anime.lif', b'annoy.fascist', b'art-fart.hum', b'b-2.jok', b'badday.hum', b'barney.txt', b'bbh_intv.txt', b'beauty.tm', b'beesherb.txt', b'bitnet.txt', b'bmdn01.txt', b'boneles2.txt', b'booze1.fun', b'butwrong.hum', b'bw-phwan.hat', b'bw.txt', b'cabbage.txt', b'caesardr.sal', b'calculus.txt', b'candy.txt', b'cartoon.law', b'cartoon.laws', b'cartoon_.txt', b'chickenheadbbs.txt', b'childhoo.jok', b'christop.int', b'clancy.txt', b'classicm.hum', b'cmu.share', b'cogdis.txt', b'collected_quotes.txt', b'commutin.jok', b'computer.txt', b'conan.txt', b'consp.txt', b'cookie.1', b'coyote.txt', b'cuchy.hum', b'cybrtrsh.txt', b'dead3.txt', b'dead4.txt', b'dead5.txt', b'deep.txt', b'dingding.hum', b'd